**ESTIMACIÓN DE LA ETP A PARTIR DE IMÁGENES DEL SENSOR MODIS**

## Descargamos los paquetes necesarios

In [ ]:
import ee
import os 
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from simpledbf import Dbf5

## Accediendo a imágenes del sensor MODIS
* Acceso a imágenes y delimitación de areas de estudio
* Descarga de datos a la computadora en formato csv
* Presentación de los datos en el mapa

In [ ]:
Map = geemap.Map()
Map.add_basemap("HYBRID")
# AOI = ee.FeatureCollection('users/Guillermo-Ortega/Limites/Departamentos')
AOI = ee.FeatureCollection('users/Guillermo-Ortega/Limites/comunNic_6487')
# Datos disponibles en reanalisis desde el 01-03-2000 a 12-03-2014
# Datos disponibles en operacional desde el 01-03-2014 a actualidad
etp = ee.ImageCollection("MODIS/006/MOD16A2")\
                      .filter(ee.Filter.calendarRange(2014, 2014, 'year'))\
                      .filter(ee.Filter.calendarRange(1,1, 'month'))\
                      .filter(ee.Filter.calendarRange(1,15, 'day_of_month'))\
                      .select('PET').median().multiply(0.1).divide(8).clip(AOI); 
# 2022 tiene datos hasta el 17 de mayo
direccion = os.path.expanduser('~/Desktop/ETP/2014')
salida_pp_stats = os.path.join(direccion, '1_1_2014.csv')
if not os.path.exists(direccion): os.makedirs(direccion)
geemap.zonal_statistics(etp, AOI, salida_pp_stats, statistics_type = 'MEDIAN', scale = 500)

vis_etp = { 'min': 0, 'max': 12, 'palette': ['9C27B0','0906f7', '15f9ff', '49fb08', 
                                             'daf895', 'f5e0f6', 'fc0419', 'B71C1C']}
vis_lim = {'color': '000000ff', 'width': 1, 'lineType': 'solid', 'fillColor': '00000000'}

Map.addLayer(etp, vis_etp, 'ept')
# Map.addLayer(AOI, vis_lim, 'Límites')
Map.centerObject(AOI, 7)
Map

In [ ]:
# si desea se puede añadir una leyenda en forma de barra con tonos degradados
colors = vis_etp["palette"]
vmin = vis_etp["min"]
vmax = vis_etp["max"]
Map.add_colorbar(vis_etp, label = "etp en mm", orientation = "vertical", position= "bottomright")

## Mostrando la distribución de la etp por departamento
* Traer el archivo desde un formato excel
* Generar histográma de cajas para realizar la comparación interdepartamental

In [ ]:
ruta = "C:/Users/guillermo.ortega/Desktop/ETP/ComETP.xlsx"
df_etp = pd.read_excel(ruta, sheet_name= "etp").sort_values("OBJECTID")
df_etp.head(3)

In [ ]:
f, ax = plt.subplots(figsize = (18, 5))
sns.boxplot(x = "Depto", y = "mar_2_2021", data = df_etp).set(
    xlabel = "Departamento", ylabel = "ETP en mm", 
    title = "ETP promedio primera quincena de enero")
plt.grid()
plt.xticks(rotation = 285)
plt.show()

In [ ]:
plt.plot(df_etp['Managua'])

## Accediendo a datos dbf

In [ ]:
# pip install simpledbf

In [ ]:
ruta_dbf = "C:/Users/guillermo.ortega/Desktop/ETP/Comunidades/ComETP.dbf"
dato_dbf = Dbf5(ruta_dbf).to_dataframe().sort_values(['OBJECTID'])
dato_dbf.head()

In [ ]:
dato_dbf[['ene_1_2001', 'ene_2_2001', 'feb_1_2001', 'feb_2_2001', 
          'mar_1_2001', 'mar_2_2001', 'abr_1_2001', 'abr_2_2001',
          'may_1_2001', 'may_2_2001', 'jun_1_2001', 'jun_2_2001',
          'jul_1_2001', 'jul_2_2001', 'ago_1_2001', 'ago_2_2001',
          'sep_1_2001', 'sep_2_2001', 'oct_1_2001', 'oct_2_2001',
          'nov_1_2001', 'nov_2_2001', 'dic_1_2001', 'dic_2_2001',]] = df_etp.iloc[:, [5,6,7,8,9,10,11,12,13,14,
                                                                                     15,16,17,18,19,20,21,22,23,
                                                                                     24,25,26,27,28]]
dato_dbf.head()